In [1]:
import pyphi

In [2]:
import numpy as np
from pyphi.distance import measures
from pyphi.distribution import flatten

pyphi.config.PARTITION_TYPE = 'TRI'
pyphi.config.SYSTEM_CUTS = 'CONCEPT_STYLE'
#pyphi.config.SYSTEM_CUTS = '3.0_STYLE'
pyphi.config.MEASURE = 'MOS' # MOS = maximum over state

@measures.register('MOS', asymmetric=True)
def mos(p, q): # q is the partitioned repertoire
    #Compute the MOS distance.
    p, q = flatten(p), flatten(q)
    info = [abs(x * np.nan_to_num(np.log(x / y))) if x > 0 else 0 for (x, y) in zip(p, q)]
    return max(info)

In [3]:
# The kenrnel needs to be restarted if you want to re-register a distance under the same name

In [4]:
tpm = pyphi.examples.basic_network().tpm
cm = pyphi.examples.basic_network().cm
state = pyphi.examples.basic_state()
node_labels = ('P', 'Q', 'R')
node_indices = (0, 1, 2)
net = pyphi.Network(tpm, cm=cm, node_labels=node_labels)
sub = pyphi.Subsystem(net, state, node_indices)

C:\Users\gfindlay\Miniconda3\envs\comp_consc_4.0\lib\site-packages\pyphi\tpm.py:46: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return tpm[conditioning_indices]


## Evaluate concepts assuming PQR is a complex

In [6]:
ces = pyphi.compute.ces(sub)

In [7]:
ces

════════════════════════════════════════════════════════════
            Cause-effect structure (4 concepts)             
════════════════════════════════════════════════════════════
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
           Concept: Mechanism = [Q], φ = 0.346574           
  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  
              MIC                         MIE               
  ┌──────────────────────────┐┌──────────────────────────┐  
  │  φ = 0.693147            ││  φ = 0.346574            │  
  │  Purview = [R]           ││  Purview = [P]           │  
  │  MIP:                    ││  MIP:                    │  
  │     ∅     ∅     Q        ││     ∅     ∅     Q        │  
  │    ─── ✕ ─── ✕ ───       ││    ─── ✕ ─── ✕ ───       │  
  │     ∅     R     ∅        ││     ∅     P     ∅        │  
  │  Repertoire:             ││  Repertoire:             │  
  │    ┌─────────────┐       ││    ┌─────────────┐       │  
  │    │ S    Pr(S)  │  

In [ ]:
# Same mechanisms

# Q purviews and states picked out are unchanged

# R cause purviews and state picked out is same
# Same future purview and picked state exists, but newly ties with another future purview PQ
# This new future purview is the one picked by default (largest).

# PQ purviews and state picked out is same

# PQR purviews and state picked out is same, including old future purview tie with PR

## Manually evaluate the CES to check for ties

In [8]:
# The place to implement this check in pyphi would be in subsustem.find_mice, line 686

In [9]:
mechanisms = pyphi.utils.powerset(sub.node_indices, nonempty=True)
cause_mips = dict()
effect_mips = dict()
for mechanism in mechanisms:
    cause_purviews = sub.potential_purviews(pyphi.Direction.CAUSE, mechanism, False)
    cause_mips[mechanism] = [sub.find_mip(pyphi.Direction.CAUSE, mechanism, purview) \
                             for purview in cause_purviews]
    effect_purviews = sub.potential_purviews(pyphi.Direction.EFFECT, mechanism, False)
    effect_mips[mechanism] = [sub.find_mip(pyphi.Direction.EFFECT, mechanism, purview) \
                              for purview in effect_purviews]

In [11]:
cause_mips[(1,)] # Q

[Repertoire irreducibility analysis
   φ = 0.693147
   Mechanism: [Q]
   Purview = [R]
   Direction: CAUSE
   Partition:
      ∅     ∅     Q 
     ─── ✕ ─── ✕ ───
      ∅     R     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1      │
     │ 1    0      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘]

In [ ]:
# No ties

In [12]:
effect_mips[(1,)] # Q

[Repertoire irreducibility analysis
   φ = 0.346574
   Mechanism: [Q]
   Purview = [P]
   Direction: EFFECT
   Partition:
      ∅     ∅     Q 
     ─── ✕ ─── ✕ ───
      ∅     P     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/4    │
     │ 1    3/4    │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0
   Mechanism: [Q]
   Purview = [R]
   Direction: EFFECT
   Partition:
      ∅     ∅     Q 
     ─── ✕ ─── ✕ ───
      ∅     R     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0.1732

In [ ]:
# No ties

In [13]:
cause_mips[(2,)] # R

[Repertoire irreducibility analysis
   φ = 0
   Mechanism: [R]
   Purview = [P]
   Direction: CAUSE
   Partition:
      ∅     ∅     R 
     ─── ✕ ─── ✕ ───
      ∅     P     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0
   Mechanism: [R]
   Purview = [Q]
   Direction: CAUSE
   Partition:
      ∅     ∅     R 
     ─── ✕ ─── ✕ ───
      ∅     Q     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0.346574
   Mec

In [ ]:
# No ties

In [14]:
effect_mips[(2,)] # R

[Repertoire irreducibility analysis
   φ = 0.346574
   Mechanism: [R]
   Purview = [P]
   Direction: EFFECT
   Partition:
      ∅     ∅     R 
     ─── ✕ ─── ✕ ───
      ∅     P     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/4    │
     │ 1    3/4    │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0.693147
   Mechanism: [R]
   Purview = [Q]
   Direction: EFFECT
   Partition:
      ∅     ∅     R 
     ─── ✕ ─── ✕ ───
      ∅     Q     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1      │
     │ 1    0      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘, Repertoire irreducibility analysis
   φ =

In [ ]:
# Tie between R/Q and R/PQ

In [15]:
cause_mips[(0,1)] # PQ

[Repertoire irreducibility analysis
   φ = 0
   Mechanism: [P, Q]
   Purview = [R]
   Direction: CAUSE
   Partition:
      ∅     P     Q 
     ─── ✕ ─── ✕ ───
      ∅     ∅     R 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1      │
     │ 1    0      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1      │
     │ 1    0      │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0.405465
   Mechanism: [P, Q]
   Purview = [Q, R]
   Direction: CAUSE
   Partition:
      ∅     P     Q 
     ─── ✕ ─── ✕ ───
      ∅     Q     R 
   Repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    0      │
     │ 10    1      │
     │ 01    0      │
     │ 11    0      │
     └──────────────┘
   Partitioned repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    1/3    │
     │ 10    2/3    │
     │ 0

In [16]:
# No ties

In [17]:
effect_mips[(0,1)] # PQ

[Repertoire irreducibility analysis
   φ = 0.693147
   Mechanism: [P, Q]
   Purview = [R]
   Direction: EFFECT
   Partition:
      ∅     ∅    P,Q
     ─── ✕ ─── ✕ ───
      ∅     R     ∅ 
   Repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    0      │
     │ 1    1      │
     └─────────────┘
   Partitioned repertoire:
     ┌─────────────┐
     │ S    Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴ │
     │ 0    1/2    │
     │ 1    1/2    │
     └─────────────┘, Repertoire irreducibility analysis
   φ = 0.346574
   Mechanism: [P, Q]
   Purview = [P, R]
   Direction: EFFECT
   Partition:
      ∅     P     Q 
     ─── ✕ ─── ✕ ───
      ∅     R     P 
   Repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    0      │
     │ 10    0      │
     │ 01    1/2    │
     │ 11    1/2    │
     └──────────────┘
   Partitioned repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    1/4    │
     │ 10    1/4    │

In [ ]:
# No ties

In [18]:
cause_mips[(0,1,2)] # PQR

[Repertoire irreducibility analysis
   φ = 0
   Mechanism: [P, Q, R]
   Purview = [Q, R]
   Direction: CAUSE
   Partition:
      ∅    P,Q    R 
     ─── ✕ ─── ✕ ───
      ∅    Q,R    ∅ 
   Repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    0      │
     │ 10    1      │
     │ 01    0      │
     │ 11    0      │
     └──────────────┘
   Partitioned repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    0      │
     │ 10    1      │
     │ 01    0      │
     │ 11    0      │
     └──────────────┘, Repertoire irreducibility analysis
   φ = 0.405465
   Mechanism: [P, Q, R]
   Purview = [P, Q, R]
   Direction: CAUSE
   Partition:
      ∅    P,R    Q 
     ─── ✕ ─── ✕ ───
      ∅    P,Q    R 
   Repertoire:
     ┌───────────────┐
     │  S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 000    0      │
     │ 100    0      │
     │ 010    0      │
     │ 110    1      │
     │ 001    0      │
     │ 101    0      │
  

In [ ]:
# No ties

In [19]:
effect_mips[(0,1,2)] # PQR

[Repertoire irreducibility analysis
   φ = 0.693147
   Mechanism: [P, Q, R]
   Purview = [P, R]
   Direction: EFFECT
   Partition:
      ∅     P    Q,R
     ─── ✕ ─── ✕ ───
      ∅     R     P 
   Repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    0      │
     │ 10    0      │
     │ 01    1      │
     │ 11    0      │
     └──────────────┘
   Partitioned repertoire:
     ┌──────────────┐
     │ S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 00    1/2    │
     │ 10    0      │
     │ 01    1/2    │
     │ 11    0      │
     └──────────────┘, Repertoire irreducibility analysis
   φ = 0.693147
   Mechanism: [P, Q, R]
   Purview = [P, Q, R]
   Direction: EFFECT
   Partition:
      ∅     P    Q,R
     ─── ✕ ─── ✕ ───
      ∅     R    P,Q
   Repertoire:
     ┌───────────────┐
     │  S     Pr(S)  │
     │ ╴╴╴╴╴╴╴╴╴╴╴╴╴ │
     │ 000    0      │
     │ 100    0      │
     │ 010    0      │
     │ 110    0      │
     │ 001    1      │
     │ 101    0 

In [ ]:
# Tie between PQR/PR and PQR/PQR

# SIA

In [ ]:
## SIA does not work, and needs implementation

In [39]:
sia = pyphi.compute.sia(sub)

In [41]:
sia

┌──────────────────────────────────────────────────────────────┐
│          System irreducibility analysis: Φ = 37/16           │
│                   Cut [Q, R] ━━/ /━━➤ [P]                    │
│                      Subsystem(P, Q, R)                      │
│ ════════════════════════════════════════════════════════════ │
│             Cause-effect structure (4 concepts)              │
│ ════════════════════════════════════════════════════════════ │
│   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   │
│              Concept: Mechanism = [Q], φ = 1/4               │
│   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   │
│               MIC                         MIE                │
│   ┌──────────────────────────┐┌──────────────────────────┐   │
│   │  φ = 1/2                 ││  φ = 1/4                 │   │
│   │  Purview = [R]           ││  Purview = [P]           │   │
│   │  MIP:                    ││  MIP:                    │   │
│   │     ∅     Q        

In [42]:
# Same system cut
# Identical mechanisms, purviews, small phi values, etc. 